In [138]:
import json
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

run bitcoind

```bash
docker run --rm -it -p 8332:8332 --name=bitcoin ruimarinho/bitcoin-core   -printtoconsole   -regtest=1   -rpcallowip=0.0.0.0/0 -rpcport=8332 -rpcpassword=bar -rpcuser=foo
```

In [139]:
rpc_user = 'foo'
rpc_password = 'bar'
url = "http://%s:%s@0.0.0.0:8332" % (rpc_user, rpc_password)

In [144]:
class Service(object):
    """Simple proxy class to AuthServiceProxy to deal with timeouts"""
    def __init__(self, url):
        self._url = url
    
    def __getattr__(self, name):
        if name.startswith('_'):
            return getattr(self, name)
        rpc = AuthServiceProxy(self._url, timeout=10000000)
        return getattr(rpc, name)

In [145]:
service = Service(url)

In [148]:
service.getinfo()

{'balance': Decimal('0E-8'),
 'blocks': 0,
 'connections': 0,
 'deprecation-warning': 'WARNING: getinfo is deprecated and will be fully removed in 0.16. Projects should transition to using getblockchaininfo, getnetworkinfo, and getwalletinfo before upgrading to 0.16',
 'difficulty': Decimal('4.656542373906925E-10'),
 'errors': '',
 'keypoololdest': 1509110883,
 'keypoolsize': 2000,
 'paytxfee': Decimal('0E-8'),
 'protocolversion': 70015,
 'proxy': '',
 'relayfee': Decimal('0.00001000'),
 'testnet': False,
 'timeoffset': 0,
 'version': 150001,
 'walletversion': 139900}

In [5]:
addr = service.getnewaddress()
addr

'mvN6LSoHnRT1U233AkX4QdxoUXeqAJoRXy'

In [6]:
n_blocks = 101
blocks = service.generatetoaddress(n_blocks, addr)

In [7]:
txs = service.listtransactions('*', 1000)

In [8]:
len(txs)

101

In [9]:
service.getblock(service.getblockhash(1))

{'bits': '207fffff',
 'chainwork': '0000000000000000000000000000000000000000000000000000000000000004',
 'confirmations': 101,
 'difficulty': Decimal('4.656542373906925E-10'),
 'hash': '2e3fcc1ed90141e4954be7e94579842e353289d20efe88c37c792050972dc252',
 'height': 1,
 'mediantime': 1508612187,
 'merkleroot': '1df308213f2f509077ebb70afaa261293f2c6529efd16900c652782f75374a2e',
 'nextblockhash': '2b1abd5b18dfbfd73bc3a13a497b758817f7511175e88437af66d80d95477b12',
 'nonce': 3,
 'previousblockhash': '0f9188f13cb7b2c71f2a335e3a4fc328bf5beb436012afca590b1a11466e2206',
 'size': 216,
 'strippedsize': 216,
 'time': 1508612187,
 'tx': ['1df308213f2f509077ebb70afaa261293f2c6529efd16900c652782f75374a2e'],
 'version': 536870912,
 'versionHex': '20000000',
 'weight': 864}

In [10]:
service.getbalance()

Decimal('50.00000000')

In [11]:
utxos = service.listunspent()
utxos

[{'account': '',
  'address': 'mvN6LSoHnRT1U233AkX4QdxoUXeqAJoRXy',
  'amount': Decimal('50.00000000'),
  'confirmations': 101,
  'safe': True,
  'scriptPubKey': '76a914a2dcdb079d2b4f7aff7e6234cedb19df80c79d9788ac',
  'solvable': True,
  'spendable': True,
  'txid': '1df308213f2f509077ebb70afaa261293f2c6529efd16900c652782f75374a2e',
  'vout': 0}]

In [12]:
addr2 = service.getnewaddress()
addr2

'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB'

In [13]:
tx = service.sendtoaddress(addr2, 10)

In [14]:
tx

'67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94'

In [15]:
tx_obj = service.gettransaction(tx)
tx_obj

{'amount': Decimal('0E-8'),
 'bip125-replaceable': 'no',
 'confirmations': 0,
 'details': [{'abandoned': False,
   'account': '',
   'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
   'amount': Decimal('-10.00000000'),
   'category': 'send',
   'fee': Decimal('-0.00004520'),
   'label': '',
   'vout': 1},
  {'account': '',
   'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
   'amount': Decimal('10.00000000'),
   'category': 'receive',
   'label': '',
   'vout': 1}],
 'fee': Decimal('-0.00004520'),
 'hex': '02000000012e4a37752f7852c60069d1ef29652c3f2961a2fa0ab7eb7790502f3f2108f31d000000006b483045022100ba00283054405a4f37ef8f0f08c57699e5c987404e83cb082b48f055d25a1e39022070f0a52c63c5c0cac2306baee8c0c2ad65ad21a040d79c182a6f48ab5e6ba03e012102e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0cfeffffff0258166bee000000001976a9148c2f3df32a942bd8477973471cb3f2850485e52a88ac00ca9a3b000000001976a91490cbfc0abba7505727702744e5a9f0304a9da22788ac65000000',
 'time': 1508612244,
 'timerece

In [16]:
service.decoderawtransaction(tx_obj['hex'])

{'hash': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
 'locktime': 101,
 'size': 226,
 'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '3045022100ba00283054405a4f37ef8f0f08c57699e5c987404e83cb082b48f055d25a1e39022070f0a52c63c5c0cac2306baee8c0c2ad65ad21a040d79c182a6f48ab5e6ba03e[ALL] 02e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0c',
    'hex': '483045022100ba00283054405a4f37ef8f0f08c57699e5c987404e83cb082b48f055d25a1e39022070f0a52c63c5c0cac2306baee8c0c2ad65ad21a040d79c182a6f48ab5e6ba03e012102e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0c'},
   'sequence': 4294967294,
   'txid': '1df308213f2f509077ebb70afaa261293f2c6529efd16900c652782f75374a2e',
   'vout': 0}],
 'vout': [{'n': 0,
   'scriptPubKey': {'addresses': ['mtJBVLkk6YbuP5fnFURSuLvnZD6EHQ5ac5'],
    'asm': 'OP_DUP OP_HASH160 8c2f3df32a942bd8477973471cb3f2850485e52a OP_EQUALVERIFY OP_CHECKSIG',
    'h

In [17]:
service.listunspent(0)

[{'address': 'mtJBVLkk6YbuP5fnFURSuLvnZD6EHQ5ac5',
  'amount': Decimal('39.99995480'),
  'confirmations': 0,
  'safe': True,
  'scriptPubKey': '76a9148c2f3df32a942bd8477973471cb3f2850485e52a88ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
  'vout': 0},
 {'account': '',
  'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
  'amount': Decimal('10.00000000'),
  'confirmations': 0,
  'safe': True,
  'scriptPubKey': '76a91490cbfc0abba7505727702744e5a9f0304a9da22788ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
  'vout': 1}]

In [18]:
service.generate(1)

['41b426d3ed47427125ae8d5494c87dc037c188a4b759c1693b80d7316d36bc82']

## Simple Raw Transaction

In [19]:
utxos = service.listunspent()
utxos

[{'account': '',
  'address': 'mvN6LSoHnRT1U233AkX4QdxoUXeqAJoRXy',
  'amount': Decimal('50.00000000'),
  'confirmations': 101,
  'safe': True,
  'scriptPubKey': '76a914a2dcdb079d2b4f7aff7e6234cedb19df80c79d9788ac',
  'solvable': True,
  'spendable': True,
  'txid': '86631a6ed4282cf2b51b0e74116e7e86ae4700b28ce54c07b14cb81a40218073',
  'vout': 0},
 {'address': 'mtJBVLkk6YbuP5fnFURSuLvnZD6EHQ5ac5',
  'amount': Decimal('39.99995480'),
  'confirmations': 1,
  'safe': True,
  'scriptPubKey': '76a9148c2f3df32a942bd8477973471cb3f2850485e52a88ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
  'vout': 0},
 {'account': '',
  'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
  'amount': Decimal('10.00000000'),
  'confirmations': 1,
  'safe': True,
  'scriptPubKey': '76a91490cbfc0abba7505727702744e5a9f0304a9da22788ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d9

In [20]:
inputs = [
    {
        'txid': utxos[0]['txid'],
        'vout': 0
    }
]
inputs

[{'txid': '86631a6ed4282cf2b51b0e74116e7e86ae4700b28ce54c07b14cb81a40218073',
  'vout': 0}]

In [21]:
outputs = {
    addr2: 49.999
}
outputs

{'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB': 49.999}

In [22]:
raw_tx = service.createrawtransaction(inputs, outputs)
raw_tx

'0200000001738021401ab84cb1074ce58cb20047ae867e6e11740e1bb5f22c28d46e1a63860000000000ffffffff01606b042a010000001976a91490cbfc0abba7505727702744e5a9f0304a9da22788ac00000000'

In [23]:
service.decoderawtransaction(raw_tx)

{'hash': 'a90eda90ab1433c3d01ccd1cf173553b93dbd0039306bec1b39fefee393e4d3d',
 'locktime': 0,
 'size': 85,
 'txid': 'a90eda90ab1433c3d01ccd1cf173553b93dbd0039306bec1b39fefee393e4d3d',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '', 'hex': ''},
   'sequence': 4294967295,
   'txid': '86631a6ed4282cf2b51b0e74116e7e86ae4700b28ce54c07b14cb81a40218073',
   'vout': 0}],
 'vout': [{'n': 0,
   'scriptPubKey': {'addresses': ['mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB'],
    'asm': 'OP_DUP OP_HASH160 90cbfc0abba7505727702744e5a9f0304a9da227 OP_EQUALVERIFY OP_CHECKSIG',
    'hex': '76a91490cbfc0abba7505727702744e5a9f0304a9da22788ac',
    'reqSigs': 1,
    'type': 'pubkeyhash'},
   'value': Decimal('49.99900000')}],
 'vsize': 85}

In [24]:
signed_resp = service.signrawtransaction(raw_tx)
signed_resp

{'complete': True,
 'hex': '0200000001738021401ab84cb1074ce58cb20047ae867e6e11740e1bb5f22c28d46e1a6386000000006b483045022100c39312263b00e8698e2ecb8acf347d5b441c9ba7db5646c0ffec560928f4edbd022037bf248bff04b34174a9a31323e93c1e4786b951300b17b9060ed175149bc965012102e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0cffffffff01606b042a010000001976a91490cbfc0abba7505727702744e5a9f0304a9da22788ac00000000'}

In [25]:
signed_raw_tx = signed_resp['hex']

In [26]:
service.decoderawtransaction(signed_raw_tx)

{'hash': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
 'locktime': 0,
 'size': 192,
 'txid': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '3045022100c39312263b00e8698e2ecb8acf347d5b441c9ba7db5646c0ffec560928f4edbd022037bf248bff04b34174a9a31323e93c1e4786b951300b17b9060ed175149bc965[ALL] 02e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0c',
    'hex': '483045022100c39312263b00e8698e2ecb8acf347d5b441c9ba7db5646c0ffec560928f4edbd022037bf248bff04b34174a9a31323e93c1e4786b951300b17b9060ed175149bc965012102e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0c'},
   'sequence': 4294967295,
   'txid': '86631a6ed4282cf2b51b0e74116e7e86ae4700b28ce54c07b14cb81a40218073',
   'vout': 0}],
 'vout': [{'n': 0,
   'scriptPubKey': {'addresses': ['mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB'],
    'asm': 'OP_DUP OP_HASH160 90cbfc0abba7505727702744e5a9f0304a9da227 OP_EQUALVERIFY OP_CHECKSIG',
    'hex

In [27]:
service.sendrawtransaction(signed_raw_tx)

'02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07'

In [28]:
service.generate(1)

['4592e769c739311202e6e2284da8afe651008db64d24b98efbec3a44923a59fe']

## Complex Raw Transaction

In [29]:
utxos = service.listunspent()
utxos

[{'account': '',
  'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
  'amount': Decimal('49.99900000'),
  'confirmations': 1,
  'safe': True,
  'scriptPubKey': '76a91490cbfc0abba7505727702744e5a9f0304a9da22788ac',
  'solvable': True,
  'spendable': True,
  'txid': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
  'vout': 0},
 {'address': 'mtJBVLkk6YbuP5fnFURSuLvnZD6EHQ5ac5',
  'amount': Decimal('39.99995480'),
  'confirmations': 2,
  'safe': True,
  'scriptPubKey': '76a9148c2f3df32a942bd8477973471cb3f2850485e52a88ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95efb18a94',
  'vout': 0},
 {'account': '',
  'address': 'mtiZxmigFzGrbZeQfW7DZ233CA2hYPGKzB',
  'amount': Decimal('10.00000000'),
  'confirmations': 2,
  'safe': True,
  'scriptPubKey': '76a91490cbfc0abba7505727702744e5a9f0304a9da22788ac',
  'solvable': True,
  'spendable': True,
  'txid': '67b8527c3931e069f2b528c7663a3210aa0ff42ba90e98146c2f4d95e

In [30]:
utxo0 = utxos[0]
utxo1 = utxos[-1]

In [31]:
secret0 = service.dumpprivkey(utxo0['address'])
secret0

'cRtt51Bc3MPbuPX2zuaJmKkK3rXi67YMKadKvUgwP2wNysGdattZ'

In [32]:
secret1 = service.dumpprivkey(utxo1['address'])
secret1

'cPU2Xp1DSXcpU6w1zDfSASHTJhVvovHf4GU7Fn5p2FuQVgsJs8a3'

In [33]:
addr0 = service.getnewaddress()
addr1 = service.getnewaddress()
addr0, addr1

('miui9FSgjza5S2EsrjwgUX79CkdsJiBrbv', 'mxs3Bqcj2B3TScGoPM6C3mFW719YNpCrWV')

In [34]:
inputs = [
    {
        'txid': utxo0['txid'],
        'vout': 0
    },
    {
        'txid': utxo1['txid'],
        'vout': 0
    }
]

inputs

[{'txid': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
  'vout': 0},
 {'txid': 'f651d0b008ca45a91a85aea54a8faed89e007670d4de0a259e4000b3da5e22a0',
  'vout': 0}]

In [35]:
outputs = {
    addr0: 49.990,
    addr1: 49.99
}

In [36]:
raw_tx = service.createrawtransaction(inputs, outputs)

In [37]:
resp = service.signrawtransaction(raw_tx, [], [secret0])

In [38]:
resp

{'complete': False,
 'errors': [{'error': 'Operation not valid with the current stack size',
   'scriptSig': '',
   'sequence': 4294967295,
   'txid': 'f651d0b008ca45a91a85aea54a8faed89e007670d4de0a259e4000b3da5e22a0',
   'vout': 0,
   'witness': []}],
 'hex': '0200000002079aec12acb34f529e5b6ba5dd15a7bf7878bbf3171d80ce3b09a05f060c2902000000006b483045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418012103141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaeadffffffffa0225edab300409e250aded47076009ed8ae8f4aa5ae851aa945ca08b0d051f60000000000ffffffff02c0aff629010000001976a91425362192e6c0abe76c2500460bcb2956552fcf2688acc0aff629010000001976a914be468aba0985c13ffc27c8060a3304e79b49531788ac00000000'}

In [39]:
signed_raw_tx = resp['hex']
service.decoderawtransaction(signed_raw_tx)

{'hash': 'c5421b444ca0867419bff61a22516cef714a128aa9a6ec3898b69904ea2fe632',
 'locktime': 0,
 'size': 267,
 'txid': 'c5421b444ca0867419bff61a22516cef714a128aa9a6ec3898b69904ea2fe632',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '3045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418[ALL] 03141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaead',
    'hex': '483045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418012103141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaead'},
   'sequence': 4294967295,
   'txid': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
   'vout': 0},
  {'scriptSig': {'asm': '', 'hex': ''},
   'sequence': 4294967295,
   'txid': 'f651d0b008ca45a91a85aea54a8faed89e007670d4de0a259e4000b3da5e22a0',
   'vout': 0}],
 'vout': [{'n': 0,
   'scriptPubKey': {

In [40]:
resp = service.signrawtransaction(signed_raw_tx, [], [secret1])
resp

{'complete': True,
 'hex': '0200000002079aec12acb34f529e5b6ba5dd15a7bf7878bbf3171d80ce3b09a05f060c2902000000006b483045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418012103141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaeadffffffffa0225edab300409e250aded47076009ed8ae8f4aa5ae851aa945ca08b0d051f6000000006a47304402207b220f198c22b52b502a045d107d8e04225000aa1e492939ff37315ea531a4d3022073488a05831c1a083b403a12cacd2d9f965e06f5790706afe07e74fbe01860a7012102e0a31ff4fc32919c96b2cd16d3d7f8e0fc4c43965ef19968e9c5048b93ac8a0cffffffff02c0aff629010000001976a91425362192e6c0abe76c2500460bcb2956552fcf2688acc0aff629010000001976a914be468aba0985c13ffc27c8060a3304e79b49531788ac00000000'}

In [41]:
signed_raw_tx = resp['hex']

In [42]:
signed_tx = service.decoderawtransaction(signed_raw_tx)
signed_tx

{'hash': '9875022e730ea8fdf1b39d02ad5f7905596715f5693763b99fd3a244d64265bd',
 'locktime': 0,
 'size': 373,
 'txid': '9875022e730ea8fdf1b39d02ad5f7905596715f5693763b99fd3a244d64265bd',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '3045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418[ALL] 03141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaead',
    'hex': '483045022100b3124f8984ac7ea4ca13433ee64104c65abc71ab924150f92a6fd5363856b6fe022069624864edd135392bd09484f2442c082a353d09e77dbe8037c721d613e25418012103141a16306b0574182937c6517d5ab5b9341285a64646451cb097c618a0fdaead'},
   'sequence': 4294967295,
   'txid': '02290c065fa0093bce801d17f3bb7878bfa715dda56b5b9e524fb3ac12ec9a07',
   'vout': 0},
  {'scriptSig': {'asm': '304402207b220f198c22b52b502a045d107d8e04225000aa1e492939ff37315ea531a4d3022073488a05831c1a083b403a12cacd2d9f965e06f5790706afe07e74fbe01860a7[ALL] 02e0a31ff4fc32919c96b2cd16d3d7

## Offline Signing

In [43]:
addr = service.getnewaddress()
addr

'mgugTiTCv5ib1kSdz7Da73Fx6VvGJ1E4wQ'

In [44]:
inputs = [
    {
        'txid': signed_tx['txid'],
        'vout': 0
    }
]
inputs

[{'txid': '9875022e730ea8fdf1b39d02ad5f7905596715f5693763b99fd3a244d64265bd',
  'vout': 0}]

In [53]:
outputs = {
    addr: 49.989
}
outputs

{'mgugTiTCv5ib1kSdz7Da73Fx6VvGJ1E4wQ': 49.989}

In [54]:
raw_tx = service.createrawtransaction(inputs, outputs)
raw_tx

'0200000001bd6542d644a2d39fb9633769f515675905795fad029db3f1fda80e732e0275980000000000ffffffff012029f529010000001976a9140f4479cff21b7f29f2e0fd7abb0c0dbf8e1f0adf88ac00000000'

In [55]:
resp = service.signrawtransaction(raw_tx)
resp

{'complete': True,
 'hex': '0200000001bd6542d644a2d39fb9633769f515675905795fad029db3f1fda80e732e027598000000006a47304402206699723ec2027a91ed88ea8dd0ff5c0a564837d8a6a80425499718bde90480e602202ce0a3ea9e12637da3aa334b57d703f83b4a83b30b690633fa2ee30d2bf5609f012102ba6cc9e14f67dc13f0a2864e05232a4fa7bc441a7c85f3f19faabecec1b4b040ffffffff012029f529010000001976a9140f4479cff21b7f29f2e0fd7abb0c0dbf8e1f0adf88ac00000000'}

In [56]:
prev_txs = [
    {
        'txid': signed_tx['txid'],
        'vout': 0,
        'scriptPubKey': signed_tx['vout'][0]['scriptPubKey']['hex']
    }
]

In [57]:
resp = service.signrawtransaction(raw_tx, prev_txs)
resp

{'complete': True,
 'hex': '0200000001bd6542d644a2d39fb9633769f515675905795fad029db3f1fda80e732e027598000000006a47304402206699723ec2027a91ed88ea8dd0ff5c0a564837d8a6a80425499718bde90480e602202ce0a3ea9e12637da3aa334b57d703f83b4a83b30b690633fa2ee30d2bf5609f012102ba6cc9e14f67dc13f0a2864e05232a4fa7bc441a7c85f3f19faabecec1b4b040ffffffff012029f529010000001976a9140f4479cff21b7f29f2e0fd7abb0c0dbf8e1f0adf88ac00000000'}

In [58]:
service.sendrawtransaction(signed_raw_tx)

'9875022e730ea8fdf1b39d02ad5f7905596715f5693763b99fd3a244d64265bd'

In [59]:
signed_raw_tx2 = resp['hex']

In [60]:
service.sendrawtransaction(signed_raw_tx2)

'56bf12a19c9d7f92083dda35a862a45780b61d2f53fca8f6de075b925c8ba95c'

## P2SH Multisig

In [61]:
addr1 = service.getnewaddress()
addr2 = service.getnewaddress()
addr3 = service.getnewaddress()
addr1, addr2, addr3

('mmqhTzHqqFkFZjRgKmuyB42XbcBzsCVYAq',
 'myQ5UDr9tpqmUW1Ekiigxks25tKZdkKDa3',
 'n2FCKrRc6H1iueTeuadCygGd6w5RLyBSMS')

In [98]:
resp = service.validateaddress(addr3)
pubkey3 = resp['pubkey']
pubkey3

'023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd02'

In [99]:
multisig = service.createmultisig(2, [addr1, addr2, pubkey3])
multisig

{'address': '2My29yMYnYhjtyAGRYP6C9bgdYkuRxSx66L',
 'redeemScript': '522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd0253ae'}

In [100]:
txid = service.sendtoaddress(multisig['address'], 10.0)
txid

'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192'

In [101]:
tx_obj = service.getrawtransaction(txid, 1)
tx_obj

{'hash': 'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192',
 'hex': '0200000001f4ee71863b47ebb0bb62e7df1038ce873b8b5f11c92a908af47b72cc9322fb61000000006a47304402205edf91a0dc950d029f92783ce1b1d93b18f3a9cdf0faf15c9769a48ea1fbcef902205d0d5bf533953469759e0cbfd23d706594a1be4c9c9c2c40ae30dee5b60f4944012102cbff63c073efe62c124d1919226e85195d018f9e0d51c1dcbf93a4c0ca8fd515feffffff0200ca9a3b0000000017a9143f57feab26bb49784ef53a1d457b7cc4a752ebe087585f3577000000001976a914eabc5353ce639d8792e0a63a28974adcb26ee4ee88ac67000000',
 'locktime': 103,
 'size': 223,
 'txid': 'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '304402205edf91a0dc950d029f92783ce1b1d93b18f3a9cdf0faf15c9769a48ea1fbcef902205d0d5bf533953469759e0cbfd23d706594a1be4c9c9c2c40ae30dee5b60f4944[ALL] 02cbff63c073efe62c124d1919226e85195d018f9e0d51c1dcbf93a4c0ca8fd515',
    'hex': '47304402205edf91a0dc950d029f92783ce1b1d93b18f3a9cdf0faf15c9769a48ea1fbcef902205d0d

In [102]:
addr4 = service.getnewaddress()
addr4

'mjjy43wHLMeoGy78Ju8DF9hBWZM1qyRss6'

In [103]:
inputs = [
    {
        'txid': tx_obj['txid'],
        'vout': 0
    }
]
inputs

[{'txid': 'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192',
  'vout': 0}]

In [104]:
outputs = {
    addr4: 9.999
}

In [105]:
raw_tx = service.createrawtransaction(inputs, outputs)
raw_tx

'02000000019231ec490bd1355c89a5e6ca86a325c78f3f77a5971473465b83b8b44e328ae90000000000ffffffff016043993b000000001976a9142e568b3aca5284dabb37107d2fdcefbb8aad851e88ac00000000'

In [106]:
secret1 = service.dumpprivkey(addr1)
secret3 = service.dumpprivkey(addr3)
secret1, secret3

('cSZzXVTZRmU3ZL2Cr5vfowNFkDUDWfv1wQ2XoxXpyd2Vdosw12HX',
 'cPKPb15uHXFHiw1xqGg2LB7duGHkLH9VcEQWvAueW1gP7ZuKxLBV')

In [107]:
prev_txs = [
    {
        'txid': tx_obj['txid'], 
        'vout': 0, 
        'scriptPubKey': tx_obj['vout'][0]['scriptPubKey']['hex'], 
        'redeemScript': multisig['redeemScript']
      }
]
prev_txs

[{'redeemScript': '522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd0253ae',
  'scriptPubKey': 'a9143f57feab26bb49784ef53a1d457b7cc4a752ebe087',
  'txid': 'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192',
  'vout': 0}]

In [108]:
resp = service.signrawtransaction(raw_tx, prev_txs, [secret1])
partial_signed_raw_tx = resp['hex']
resp

{'complete': False,
 'errors': [{'error': 'Operation not valid with the current stack size',
   'scriptSig': '0047304402206280be9dde8556973b952eb4fb8c3b15b77d2497bb90e14aedc9c065e47f4d430220519c14edf09c72f2dbca4c258b927d0f50f06777b62403b6a47a676871594b46014c69522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd0253ae',
   'sequence': 4294967295,
   'txid': 'e98a324eb4b8835b46731497a5773f8fc725a386cae6a5895c35d10b49ec3192',
   'vout': 0,
   'witness': []}],
 'hex': '02000000019231ec490bd1355c89a5e6ca86a325c78f3f77a5971473465b83b8b44e328ae900000000b40047304402206280be9dde8556973b952eb4fb8c3b15b77d2497bb90e14aedc9c065e47f4d430220519c14edf09c72f2dbca4c258b927d0f50f06777b62403b6a47a676871594b46014c69522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c

In [110]:
resp = service.signrawtransaction(partial_signed_raw_tx, prev_txs, [secret3])
signed_raw = resp['hex']
resp

{'complete': True,
 'hex': '02000000019231ec490bd1355c89a5e6ca86a325c78f3f77a5971473465b83b8b44e328ae900000000fc0047304402206280be9dde8556973b952eb4fb8c3b15b77d2497bb90e14aedc9c065e47f4d430220519c14edf09c72f2dbca4c258b927d0f50f06777b62403b6a47a676871594b460147304402206eb27e17f0dcba7d5a2c37a87c65adce5cbd3229d57dcbbfc161b5aa717860400220437986cab2360d7d33f066385e10b380616d7be38f185e29a3f4fdbce3b77f40014c69522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd0253aeffffffff016043993b000000001976a9142e568b3aca5284dabb37107d2fdcefbb8aad851e88ac00000000'}

In [114]:
txid = service.sendrawtransaction(signed_raw)
txid

'497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952'

In [116]:
service.getrawtransaction(txid, 1)

{'hash': '497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952',
 'hex': '02000000019231ec490bd1355c89a5e6ca86a325c78f3f77a5971473465b83b8b44e328ae900000000fc0047304402206280be9dde8556973b952eb4fb8c3b15b77d2497bb90e14aedc9c065e47f4d430220519c14edf09c72f2dbca4c258b927d0f50f06777b62403b6a47a676871594b460147304402206eb27e17f0dcba7d5a2c37a87c65adce5cbd3229d57dcbbfc161b5aa717860400220437986cab2360d7d33f066385e10b380616d7be38f185e29a3f4fdbce3b77f40014c69522102482afcd14a3775d03b74f3008fa57ad9800bfdf97f44299ee109c2a8262ad98121035c6149e6591c323531d567983cde95830928c9133b1828c6e81abf2d1b66025221023c683dd2a4cff63fe42c1c7d6fbca3954239a3e445b8c377952084a33d57dd0253aeffffffff016043993b000000001976a9142e568b3aca5284dabb37107d2fdcefbb8aad851e88ac00000000',
 'locktime': 0,
 'size': 337,
 'txid': '497405713e6c094663245d28f465045e982cfece36d9b5a24bbcd9c68bad7952',
 'version': 2,
 'vin': [{'scriptSig': {'asm': '0 304402206280be9dde8556973b952eb4fb8c3b15b77d2497bb90e14aedc9c065e47f4d430220519c1

## Debug

In [118]:
service.getrawtransaction('16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e', 1)

{'hash': '16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e',
 'hex': '01000000015279ad8bc6d9bc4ba2b5d936cefe2c985e0465f4285d246346096c3e71057449000000006a47304402205d0e0daf0d3733c9fa787dbe2455a6b927408d59ffc39efe65bb671a1ce93b1502205896d19ac9b154fd38a167f06e55d432af6aa80c3130ca40f32cc75993df3f14012102ff97629efcb01518a6a5a41a40c6b2fbb93e9c726ebfe84b2b022189b71e4a2c01000000018033023b000000002a046ac0ef59b1752102b568858a407a8721923b89df9963d30013639ac690cce5f555529b77b83cbfc7ac00000000',
 'locktime': 0,
 'size': 208,
 'txid': '16ae89fec61792e9015bfb3fced5f296e4c3b71c36e1cbe00266ffd4e9a5c46e',
 'version': 1,
 'vin': [{'scriptSig': {'asm': '304402205d0e0daf0d3733c9fa787dbe2455a6b927408d59ffc39efe65bb671a1ce93b1502205896d19ac9b154fd38a167f06e55d432af6aa80c3130ca40f32cc75993df3f14[ALL] 02ff97629efcb01518a6a5a41a40c6b2fbb93e9c726ebfe84b2b022189b71e4a2c',
    'hex': '47304402205d0e0daf0d3733c9fa787dbe2455a6b927408d59ffc39efe65bb671a1ce93b1502205896d19ac9b154fd38a167f06e55d432af6a

In [136]:
_ = service.generate(100)

In [149]:
service.gettransaction('cacb91e8ddab354d0a67016859d57700ed691da8bd3587fe3b8fb0747608338a')

JSONRPCException: -5: Invalid or non-wallet transaction id

In [132]:
service.getinfo()

{'balance': Decimal('5230.10000000'),
 'blocks': 205,
 'connections': 0,
 'deprecation-warning': 'WARNING: getinfo is deprecated and will be fully removed in 0.16. Projects should transition to using getblockchaininfo, getnetworkinfo, and getwalletinfo before upgrading to 0.16',
 'difficulty': Decimal('4.656542373906925E-10'),
 'errors': '',
 'keypoololdest': 1508612144,
 'keypoolsize': 1999,
 'paytxfee': Decimal('0E-8'),
 'protocolversion': 70015,
 'proxy': '',
 'relayfee': Decimal('0.00001000'),
 'testnet': False,
 'timeoffset': 0,
 'version': 150001,
 'walletversion': 139900}

In [133]:
service.getblock(service.getblockhash(205))

{'bits': '207fffff',
 'chainwork': '000000000000000000000000000000000000000000000000000000000000019c',
 'confirmations': 1,
 'difficulty': Decimal('4.656542373906925E-10'),
 'hash': '4c3523ed9bd48c2d99a4cf2a900906d3f2964853ac3a1bc0e03fcd4eb2fb8d7b',
 'height': 205,
 'mediantime': 1508897661,
 'merkleroot': 'a7a98c584fc5601dd2b90c57f9f391cead06cf5f2775be1f482b49cabc8aa3a8',
 'nonce': 0,
 'previousblockhash': '1d059278faa6f93d16f47447bbb47cbb2cf6430ffc78dce022429c5ea22d9581',
 'size': 228,
 'strippedsize': 228,
 'time': 1508897662,
 'tx': ['a7a98c584fc5601dd2b90c57f9f391cead06cf5f2775be1f482b49cabc8aa3a8'],
 'version': 805306371,
 'versionHex': '30000003',
 'weight': 912}